In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd


In [2]:
# list_csv=["data_d1/EURUSD1440_data.csv","data_d1/GBPUSD1440_data.csv","data_d1/USDCAD1440_data.csv","data_d1/USDCHF1440_data.csv","data_d1/USDJPY1440_data.csv"]
# list_csv=["data_h1/EURUSD60_data.csv","data_h1/GBPUSD60_data.csv","data_h1/USDCAD60_data.csv","data_h1/USDCHF60_data.csv","data_h1/USDJPY60_data.csv"]
list_csv=["data_m5/EURUSD5_data.csv","data_m5/GBPUSD5_data.csv","data_m5/USDCAD5_data.csv","data_m5/USDCHF5_data.csv","data_m5/USDJPY5_data.csv"]

In [3]:
from vungo.reports import full


In [4]:
# Function to create sequences for LSTM
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), :]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)
# Adjusted function to store and return trade results after each trade, including a 5% transaction fee
def simple_trade_with_results_and_fees(test_data, predicted_data, trade_size=100, transaction_fee_percent=0.001):
    trade_results = []
    total_profit = 10000

    for i in range(len(predicted_data)):
        # Assuming 'adjcp' is the adjusted closing price
        real_price = test_data.iloc[i]['adjcp']
        predicted_price = predicted_data[i]

        # If this is not the last row, take the opening price of the next row as the exit price
        if i < len(test_data) - 1:
            exit_price = test_data.iloc[i + 1]['open']
        else:
            # If it is the last row, a trade cannot be executed
            break

        # Determine the action based on the prediction and calculate the gross profit or loss
        if predicted_price > real_price:
            # Predicting an increase in price, buy and then sell at the next opening price
            gross_profit = (exit_price - real_price) * trade_size
            trade_action = 'Buy'
        else:
            # Predicting a decrease in price, sell and then buy back at the next opening price
            gross_profit = (real_price - exit_price) * trade_size
            trade_action = 'Sell'

        # Calculate the transaction fee
        transaction_fee = trade_size * transaction_fee_percent

        # Net profit after the transaction fee
        net_profit = gross_profit - transaction_fee

        # Update total net profit
        total_profit += net_profit

        # Store the trade result
        trade_results.append({
            'Trade Number': i+1,
            'Action': trade_action,
            'Entry Price': real_price,
            'Exit Price': exit_price,
            'Predicted Price': predicted_price,
            'Gross Profit/Loss': gross_profit,
            'Transaction Fee': transaction_fee,
            'Net Profit/Loss': net_profit,
            "Account balance":total_profit
        })
    return total_profit, trade_results

In [5]:
model_name="LSTM"
for pair in list_csv:
    tic_name= pair.split("/")[1].split(".")[0].split("_")[0]
    period = pair.split("/")[0].split("_")[1]
    data=pd.read_csv(pair)
    # Selecting relevant features for the LSTM model
    features = ['adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi', 'ema', 'boll']
    data_lstm = data[features]
        
    # Scaling the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_lstm)
    # Scaling the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_lstm)



    # Time step for LSTM sequence
    time_step = 60

    # Create the dataset for LSTM
    X, y = create_dataset(scaled_data, time_step)

    # Splitting the dataset into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=1, verbose=1)

    # Predicting with the model
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    # Create dummy arrays with the same number of rows as train_predict and test_predict
    test_dummy = np.zeros(shape=(len(test_predict), scaled_data.shape[1]))
    # Replace the first column of the dummy arrays with the predictions
    test_dummy[:, 0] = test_predict[:, 0]
    # Apply inverse_transform using these dummy arrays
    test_predict_original_scale = scaler.inverse_transform(test_dummy)[:, 0]
    # Calculating RMSE performance metrics for the model on the original scale
    y_test_original = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((len(y_test), 8))), axis=1))[:, 0]


    total_profit, trade_results = simple_trade_with_results_and_fees(
        data.iloc[len(X_train):(len(X_train)+len(X_test))],
        test_predict_original_scale
    )
    trade_results_df = pd.DataFrame(trade_results)
    start_date = pd.to_datetime('2020-01-01')

    # Now we'll create a date range starting from the 'start_date' with the number of periods equal to the number of trades.
    trade_dates = pd.date_range(start=start_date, periods=len(trade_results_df), freq='B')  # 'B' denotes business day frequency

    # Assign this date range to a new column in the DataFrame
    trade_results_df['Date'] = trade_dates

    # Set this new 'Date' column as the index of the DataFrame
    trade_results_df.set_index('Date', inplace=True)

    # Rename the index to 'date' for compatibility with QuantStats
    trade_results_df.index.name = 'date'

    # Now let's proceed to convert the trade results to returns and apply QuantStats.
    # Calculate daily returns from the 'Net Profit/Loss' column
    # Convert Net Profit/Loss to returns by dividing by the account balance of the previous day
    trade_results_df['Daily Returns'] = trade_results_df['Net Profit/Loss'] / trade_results_df['Account balance'].shift(1)
    trade_results_df['Daily Returns'].fillna(0, inplace=True)  # Replace NaN values with 0 for the first day

    # Convert the daily returns to a pandas Series with the date as the index
    daily_returns = trade_results_df['Daily Returns']
    
    # Extend pandas functionality with metrics, etc.
    full(daily_returns, tic_name=tic_name, model_name=model_name)
        # Mở file CSV ở chế độ ghi và ghi các giá trị vào file
    import csv
    # Đường dẫn đến file CSV
    csv_file_path = f'{tic_name}-{model_name}.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Ghi các giá trị vào file dưới dạng hàng mới
        writer.writerow(['Action Sell', 0])
        writer.writerow(['Action Buy', 0])
        writer.writerow(['Total Reward', 0])
        writer.writerow(['Total Profit', total_profit])
        writer.writerow(['Ticket', tic_name])
        writer.writerow(['Model', model_name])

    print(f"Đã ghi thông tin vào {csv_file_path}")

118/118 [==============================] - 2s 19ms/step
Đã ghi thông tin vào EURUSD5-LSTM.csv
118/118 [==============================] - 2s 14ms/step
Đã ghi thông tin vào GBPUSD5-LSTM.csv
118/118 [==============================] - 2s 15ms/step
Đã ghi thông tin vào USDCAD5-LSTM.csv
118/118 [==============================] - 2s 14ms/step
Đã ghi thông tin vào USDCHF5-LSTM.csv
118/118 [==============================] - 2s 16ms/step
Đã ghi thông tin vào USDJPY5-LSTM.csv


In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import GRU, Dense
import numpy as np  
model_name="GRU"
for pair in list_csv:
    tic_name= pair.split("/")[1].split(".")[0].split("_")[0]
    period = pair.split("/")[0].split("_")[1]
    data=pd.read_csv(pair)
    # Selecting relevant features for the GRU model
    features = ['adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi', 'ema', 'boll']
    data_GRU = data[features]
        
    # Scaling the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_GRU)
    # Scaling the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_GRU)



    # Time step for GRU sequence
    time_step = 60

    # Create the dataset for GRU
    X, y = create_dataset(scaled_data, time_step)

    # Splitting the dataset into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # GRU model
    model = Sequential()
    model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(GRU(units=50))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=1, verbose=1)

    # Predicting with the model
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    # Create dummy arrays with the same number of rows as train_predict and test_predict
    test_dummy = np.zeros(shape=(len(test_predict), scaled_data.shape[1]))
    # Replace the first column of the dummy arrays with the predictions
    test_dummy[:, 0] = test_predict[:, 0]
    # Apply inverse_transform using these dummy arrays
    test_predict_original_scale = scaler.inverse_transform(test_dummy)[:, 0]
    # Calculating RMSE performance metrics for the model on the original scale
    y_test_original = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((len(y_test), 8))), axis=1))[:, 0]


    total_profit, trade_results = simple_trade_with_results_and_fees(
        data.iloc[len(X_train):(len(X_train)+len(X_test))],
        test_predict_original_scale
    )
    trade_results_df = pd.DataFrame(trade_results)
    start_date = pd.to_datetime('2020-01-01')

    # Now we'll create a date range starting from the 'start_date' with the number of periods equal to the number of trades.
    trade_dates = pd.date_range(start=start_date, periods=len(trade_results_df), freq='B')  # 'B' denotes business day frequency

    # Assign this date range to a new column in the DataFrame
    trade_results_df['Date'] = trade_dates

    # Set this new 'Date' column as the index of the DataFrame
    trade_results_df.set_index('Date', inplace=True)

    # Rename the index to 'date' for compatibility with QuantStats
    trade_results_df.index.name = 'date'

    # Now let's proceed to convert the trade results to returns and apply QuantStats.
    # Calculate daily returns from the 'Net Profit/Loss' column
    # Convert Net Profit/Loss to returns by dividing by the account balance of the previous day
    trade_results_df['Daily Returns'] = trade_results_df['Net Profit/Loss'] / trade_results_df['Account balance'].shift(1)
    trade_results_df['Daily Returns'].fillna(0, inplace=True)  # Replace NaN values with 0 for the first day

    # Convert the daily returns to a pandas Series with the date as the index
    daily_returns = trade_results_df['Daily Returns']
    
    # Extend pandas functionality with metrics, etc.
    full(daily_returns, tic_name=tic_name, model_name=model_name)
        # Mở file CSV ở chế độ ghi và ghi các giá trị vào file
    import csv
    # Đường dẫn đến file CSV
    csv_file_path = f'{tic_name}-{model_name}.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Ghi các giá trị vào file dưới dạng hàng mới
        writer.writerow(['Action Sell', 0])
        writer.writerow(['Action Buy', 0])
        writer.writerow(['Total Reward', 0])
        writer.writerow(['Total Profit', total_profit])
        writer.writerow(['Ticket', tic_name])
        writer.writerow(['Model', model_name])

    print(f"Đã ghi thông tin vào {csv_file_path}")

118/118 [==============================] - 2s 14ms/step
Đã ghi thông tin vào EURUSD5-GRU.csv
118/118 [==============================] - 3s 21ms/step
Đã ghi thông tin vào GBPUSD5-GRU.csv
118/118 [==============================] - 2s 15ms/step
Đã ghi thông tin vào USDCAD5-GRU.csv
118/118 [==============================] - 2s 15ms/step
Đã ghi thông tin vào USDCHF5-GRU.csv
118/118 [==============================] - 2s 17ms/step
Đã ghi thông tin vào USDJPY5-GRU.csv


In [11]:
total_profit

48247.8000000001